In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import PIL
from keras import layers, optimizers
from keras.models import Model, model_from_json, Sequential
from keras.layers import Input, Activation, Dense, Conv2D, Flatten, ZeroPadding2D, BatchNormalization
from keras.layers import MaxPooling2D, AveragePooling2D, Dropout, GlobalMaxPooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam,SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from random import seed 
from random import randint

Using TensorFlow backend.


In [2]:
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import keras.backend as K
K.set_image_data_format('channels_last')
from PIL import Image


directory = os.path.abspath("t_train")
print(directory)

X = []
Y = []
i = 0
category = ['Cat', 'Dog', 'Chick']
for c in category:
    path = os.path.join(directory, c)
    for file in os.listdir(path):
        classes = category.index(c)
        Y.append(classes)
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img_r = cv2.resize(img, (224, 224))
        X.append(img_r)
        
X_train = np.array(X)
Y_train = np.array(Y)
Y_train = to_categorical(Y_train)
cv2.imshow("New Image", X_train[2])
cv2.waitKey(0)
cv2.destroyAllWindows()

np.random.seed(1)
m = X_train.shape[0]
permutation = list(np.random.permutation(m))
print("Permutations: ", permutation)
X = X_train[permutation, :]
Y = Y_train[permutation, :]
X_train = X
Y_train = Y
m = X_train.shape[0]
for i in range(m):
    cv2.imshow("a"+str(i), X_train[i])
print("Y_train corresponding to X_train: ",Y_train)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
from keras.applications import VGG16
#vgg_conv = VGG16(weights = 'imagenet', include_top = True)

vgg_conv.summary()

In [6]:
input_tensor = Input((224, 224, 3))
vgg = VGG16(weights = 'imagenet', include_top = False, input_tensor = input_tensor)
print(vgg.summary())











_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         


In [7]:
for l in vgg.layers[:-4]:
    l.trainable = False
for layer in vgg.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x0000020E75804B08> False
<keras.layers.convolutional.Conv2D object at 0x0000020E73891108> False
<keras.layers.convolutional.Conv2D object at 0x0000020E75804AC8> False
<keras.layers.pooling.MaxPooling2D object at 0x0000020E75869B08> False
<keras.layers.convolutional.Conv2D object at 0x0000020E73972C08> False
<keras.layers.convolutional.Conv2D object at 0x0000020E738C5108> False
<keras.layers.pooling.MaxPooling2D object at 0x0000020E75888BC8> False
<keras.layers.convolutional.Conv2D object at 0x0000020E7588FD08> False
<keras.layers.convolutional.Conv2D object at 0x0000020E7589BFC8> False
<keras.layers.convolutional.Conv2D object at 0x0000020E75896E08> False
<keras.layers.pooling.MaxPooling2D object at 0x0000020E758A8D88> False
<keras.layers.convolutional.Conv2D object at 0x0000020E758B2448> False
<keras.layers.convolutional.Conv2D object at 0x0000020E758B66C8> False
<keras.layers.convolutional.Conv2D object at 0x0000020E758BEB08> False
<ker

In [8]:
X = vgg.layers[-1].output
X = Flatten()(X)
X = Dense(256, activation = 'relu')(X)
X = Dense(256, activation = 'relu')(X)
X = Dense(3, activation = 'softmax')(X)
model_vgg = Model(vgg.input, X)
print(model_vgg.summary())
for layer in model_vgg.layers:
    print(layer, layer.trainable)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 20,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_batchsize = 100
validation_batchsize = 10

train_generator = train_datagen.flow_from_directory('Transfer_Train',
                                 target_size = (224, 224),
                                 batch_size = train_batchsize,
                                 class_mode = 'categorical',
                                 )

validation_generator = validation_datagen.flow_from_directory('Transfer_vali',
                                 target_size = (224, 224),
                                 batch_size = validation_batchsize,
                                 class_mode = 'categorical',
                                 shuffle = False,
                                 )

Found 297 images belonging to 3 classes.
Found 89 images belonging to 3 classes.


In [13]:
model_vgg.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr = 1e-2), metrics = ['accuracy'])
history = model_vgg.fit_generator(train_generator,
                              steps_per_epoch=train_generator.samples/train_generator.batch_size,
                              epochs = 10,
                              validation_data = validation_generator,
                              validation_steps=validation_generator.samples/validation_generator.batch_size,
                              verbose = 1)
#model_vgg.fit(X_train, Y_train, epochs = 10)

Epoch 1/10
3/2 [==============================] - 294s 98s/step - loss: 1.1065 - acc: 0.3503 - val_loss: 1.1035 - val_acc: 0.3371
Epoch 2/10
3/2 [==============================] - 271s 90s/step - loss: 1.0999 - acc: 0.3534 - val_loss: 1.0985 - val_acc: 0.3371
Epoch 3/10
3/2 [==============================] - 275s 92s/step - loss: 1.0987 - acc: 0.3464 - val_loss: 1.0989 - val_acc: 0.3371
Epoch 4/10
3/2 [==============================] - 273s 91s/step - loss: 1.1018 - acc: 0.2961 - val_loss: 1.0987 - val_acc: 0.3371
Epoch 5/10
3/2 [==============================] - 274s 91s/step - loss: 1.0979 - acc: 0.3605 - val_loss: 1.0987 - val_acc: 0.3371
Epoch 6/10
3/2 [==============================] - 269s 90s/step - loss: 1.0986 - acc: 0.3466 - val_loss: 1.0986 - val_acc: 0.3371
Epoch 7/10
3/2 [==============================] - 270s 90s/step - loss: 1.0997 - acc: 0.3230 - val_loss: 1.0985 - val_acc: 0.3371
Epoch 8/10
3/2 [==============================] - 267s 89s/step - loss: 1.0979 - acc: 0.34

In [11]:
#model_vgg.predict(X_train)

In [12]:
#print(Y_train)